<a href="https://colab.research.google.com/github/AndresMontesDeOca/Redes_Neuronales/blob/main/06_Redes_Neuronales_Conv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries and Data Load

In [23]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import NumpyArrayIterator


ColabNotebook = 'google.colab' in str(get_ipython())

if ColabNotebook:
    # monta G-drive en entorno COLAB
    from google.colab import drive
    drive.mount('/content/drive/')

    # carpeta donde se encuentran archivos .py auxiliares
    FUENTES_DIR = '/content/drive/MyDrive/Colab Notebooks/Redes_Neuronales/Fuentes/'
    DATOS_DIR = '/content/drive/MyDrive/Colab Notebooks/Redes_Neuronales/Data/'      # carpeta donde se encuentran los datasets
else:
    # configuración para notebook con instalación LOCAL
    FUENTES_DIR = '../Fuentes'         # carpeta donde se encuentran archivos .py auxiliares
    DATOS_DIR   = '../Datos/' # carpeta donde se encuentran los datasets

# agrega ruta de busqueda donde tenemos archivos .py
import sys
sys.path.append(FUENTES_DIR)

################################################################
def plot_history(history, start_epoch=0, metrics=None):
    if isinstance(metrics, str):
        metrics = [metrics]

    if metrics is None:
        metrics = [x for x in history.history.keys() if x[:4] != 'val_']

    if len(metrics) == 0:
        print('No metrics to display.')
        return

    # Get the epochs and filter them starting from start_epoch
    x = history.epoch[start_epoch:]

    rows = 1
    cols = len(metrics)
    count = 0

    plt.figure(figsize=(12 * cols, 8))

    for metric in sorted(metrics):
        count += 1
        plt.subplot(rows, cols, count)
        plt.plot(x, history.history[metric][start_epoch:], label='Train')
        val_metric = f'val_{metric}'
        if val_metric in history.history.keys():
            plt.plot(x, history.history[val_metric][start_epoch:], label='Validation')
        plt.title(metric.capitalize())
        plt.legend()
    plt.show()
################################################################

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## 01 - Ejercicio MINST

In [ ]:
# import time
# import numpy as np
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense
# from tensorflow.keras.regularizers import l1, l2

# from tensorflow.keras.datasets import mnist
# (X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# # Redimensionar los datos para que sean compatibles con MinMaxScaler
# X_train = X_train.reshape(-1, 28 * 28)
# X_test = X_test.reshape(-1, 28 * 28)

# # 2. Normalizar los valores de las imágenes a un rango de 0 a 1 utilizando MinMaxScaler
# scaler = MinMaxScaler()
# X_train = scaler.fit_transform(X_train)
# X_test = scaler.transform(X_test)

# # Volver a dar forma a los datos para que sean compatibles con la entrada de Conv2D
# X_train = X_train.reshape(-1, 28, 28, 1)
# X_test = X_test.reshape(-1, 28, 28, 1)

# # Convertir las etiquetas a one-hot encoding
# Y_train = to_categorical(Y_train, 10)
# Y_test = to_categorical(Y_test, 10)

# # 3. Construir el modelo CNN
# model = Sequential([
#     Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
#     MaxPooling2D(pool_size=(2, 2)),
#     Conv2D(64, kernel_size=(3, 3), activation='relu'),
#     MaxPooling2D(pool_size=(2, 2)),
#     Flatten(),
#     Dense(64, activation='relu'),
#     Dense(10, activation='softmax')
# ])

# patience = 5

# # Compilar el modelo
# model.compile(
#     optimizer='adam',
#     loss='categorical_crossentropy',
#     metrics=['accuracy'])

# early_stop = EarlyStopping(
#     monitor='val_loss',
#     patience=patience,
#     restore_best_weights=True)



# # 4. Entrenar el modelo
# model.fit(
#     X_train,
#     Y_train,
#     validation_data=(X_test, Y_test),
#     epochs=10,
#     callbacks=[early_stop],
#     verbose=1,
#     batch_size=200)

# plot_history(model.history)
# # 5. Guardar el modelo
# model.save('mnist_cnn_model.h5')

# print("Modelo entrenado y guardado como 'mnist_cnn_model.h5'")

# # model.evaluate(X_test, Y_test)

In [ ]:
# # Evaluar el modelo en el conjunto de prueba
# loss, accuracy = model.evaluate(X_test, Y_test, verbose=1)

# print(f"Pérdida en el conjunto de prueba: {loss:.4f}")
# print(f"Precisión en el conjunto de prueba: {accuracy:.4f}")

## 03 Fingers

### Auxiliary Functions

In [49]:
#############################################################################
def get_data(url, carpeta_destino):
    """
    Descarga y extrae un archivo ZIP desde una URL a una carpeta de destino.

    Parámetros:
    url (str): La URL del archivo ZIP a descargar.
    carpeta_destino (str): La ruta de la carpeta donde se guardará y extraerá el archivo ZIP.

    Devuelve:
    str: La ruta del directorio donde se extrajo el contenido del archivo ZIP.
    """
    # Crear la carpeta de destino si no existe
    os.makedirs(carpeta_destino, exist_ok=True)

    # Descargar el archivo ZIP
    response = requests.get(url)
    zip_path = os.path.join(carpeta_destino, os.path.basename(url))

    # Guardar el archivo ZIP descargado
    with open(zip_path, 'wb') as f:
        f.write(response.content)

    # Extraer el contenido del archivo ZIP
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(carpeta_destino)

        # Obtener el nombre del directorio extraído
        extracted_items = zip_ref.namelist()
        common_prefix = os.path.commonprefix(extracted_items)

        # Si el common_prefix es una subcarpeta, aseguramos obtener solo el primer nivel
        if not os.path.isdir(os.path.join(carpeta_destino, common_prefix)):
            common_prefix = common_prefix.split('/')[0]

        extracted_dir_path = os.path.join(carpeta_destino, common_prefix)

    # Normalizar la ruta para asegurar compatibilidad con Windows
    extracted_dir_path = os.path.normpath(extracted_dir_path)

    # Eliminar el archivo ZIP descargado (opcional)
    # os.remove(zip_path)

    # Devolver el path donde se extrajo el ZIP
    return extracted_dir_path
#############################################################################

#############################################################################
def show_images_random(carpeta_raiz,  num_imagenes=1, subcarpeta='train'):
    """
    Muestra un número específico de imágenes al azar de una estructura de carpetas especificada, junto con sus dimensiones.

    Args:
        carpeta_raiz (str): Ruta a la carpeta raíz que contiene las subcarpetas de imágenes.
        subcarpeta (str): Nombre de la subcarpeta dentro de la carpeta raíz donde se encuentran las imágenes.
                          Por defecto es 'train'.
        num_imagenes (int): Número de imágenes a mostrar por subcarpeta. Por defecto es 1.

    Raises:
        ValueError: Si la carpeta especificada no existe.

    Returns:
        None

    Example:
        show_images_random('/ruta/a/dataset', 'train', 3)
    """
    # Construir la ruta completa a la subcarpeta 'train'
    carpeta = os.path.join(carpeta_raiz, subcarpeta)

    # Verificar si la carpeta existe
    if not os.path.isdir(carpeta):
        raise ValueError(f"La carpeta '{carpeta}' no existe.")

    # Obtener una lista de todas las subcarpetas en 'train'
    subcarpetas = [d for d in os.listdir(carpeta) if os.path.isdir(os.path.join(carpeta, d))]

    dimensiones = []

    for subcarpeta in subcarpetas:
        ruta_subcarpeta = os.path.join(carpeta, subcarpeta)

        # Obtener una lista de todos los archivos en la subcarpeta
        archivos = [f for f in os.listdir(ruta_subcarpeta) if os.path.isfile(os.path.join(ruta_subcarpeta, f))]

        # Filtrar solo las imágenes (asumiendo formatos comunes)
        imagenes = [f for f in archivos if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif', '.tiff'))]

        # Verificar si hay imágenes en la subcarpeta
        if not imagenes:
            print(f"No hay imágenes en la subcarpeta '{ruta_subcarpeta}'")
            continue

        # Seleccionar las imágenes al azar
        imagenes_seleccionadas = random.sample(imagenes, min(num_imagenes, len(imagenes)))

        for imagen_seleccionada in imagenes_seleccionadas:
            # Abrir la imagen y obtener sus dimensiones
            ruta_imagen = os.path.join(ruta_subcarpeta, imagen_seleccionada)
            with Image.open(ruta_imagen) as img:
                ancho, alto = img.size
                profundidad = img.mode
                dimensiones.append((imagen_seleccionada, subcarpeta, alto, ancho, profundidad, ruta_imagen))

    # Mostrar las imágenes y sus dimensiones
    if dimensiones:  # Check if any images were found
      fig, axs = plt.subplots(1, len(dimensiones), figsize=(15, 5))
    else:
      print("No se encontraron imágenes en la carpeta especificada.")

    if len(dimensiones) == 1:
      fig, axs = plt.subplots() # Create a single subplot
      axs = [axs]
    else:
      fig, axs = plt.subplots(1, num_imagenes, figsize=(20, 5)) # Create multiple subplots

    for ax, (nombre_imagen, nombre_carpeta, alto, ancho, profundidad, ruta_imagen) in zip(axs, dimensiones):
        img = Image.open(ruta_imagen)
        ax.imshow(img)
        ax.set_title(f"Archivo: {nombre_imagen}\nCarpeta: {nombre_carpeta}\nAlto: {alto}, Ancho: {ancho}, Profundidad: {profundidad}")
        ax.axis('off')

    plt.show()
#############################################################################
def count_files_in_subfolders(path):
    file_counts = {}

    for root, dirs, files in os.walk(path):
        for subdir in dirs:
            subdir_path = os.path.join(root, subdir)
            file_count = sum([len(files) for r, d, files in os.walk(subdir_path)])
            file_counts[subdir_path] = file_count

    return file_counts
#############################################################################
#############################################################################
def split_datapath(base_path, val_percentage):
    """
    Divide un conjunto de datos en conjuntos de entrenamiento y validación.

    Args:
        base_path (str): Ruta a la carpeta base que contiene las subcarpetas de clases.
        val_percentage (float): Porcentaje de imágenes que se deben mover a la carpeta de validación.

    Returns:
        None
    """
    classes = [d for d in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, d))]
    train_path = os.path.join(base_path, 'train')
    val_path = os.path.join(base_path, 'validation')

    os.makedirs(train_path, exist_ok=True)
    os.makedirs(val_path, exist_ok=True)

    for cls in classes:
        class_path = os.path.join(base_path, cls)
        if not os.path.isdir(class_path):
            continue

        images = os.listdir(class_path)
        random.shuffle(images)
        split_idx = int(len(images) * (1 - val_percentage))

        train_images = images[:split_idx]
        val_images = images[split_idx:]

        train_class_path = os.path.join(train_path, cls)
        val_class_path = os.path.join(val_path, cls)

        os.makedirs(train_class_path, exist_ok=True)
        os.makedirs(val_class_path, exist_ok=True)

        for img in train_images:
            shutil.move(os.path.join(class_path, img), os.path.join(train_class_path, img))

        for img in val_images:
            shutil.move(os.path.join(class_path, img), os.path.join(val_class_path, img))

        # Eliminar la carpeta de clase original si está vacía
        if not os.listdir(class_path):
            os.rmdir(class_path)

    print("División del conjunto de datos completada.")
#############################################################################

In [21]:
def verificar_normalizacion(data_directory, data_type, image_size, sample_size=5):
    # Tomar una muestra de imágenes para verificar si ya están normalizadas
    sample_images = []
    for root, _, files in os.walk(f'{data_directory}/{data_type}'):
        for file in files:
            if file.endswith(('jpg', 'jpeg', 'png')):
                img_path = os.path.join(root, file)
                img = load_img(img_path, target_size=image_size)
                img_array = img_to_array(img)
                sample_images.append(img_array)
                if len(sample_images) >= sample_size:
                    break
        if len(sample_images) >= sample_size:
            break
    sample_images = np.array(sample_images)

    # Verificar si los valores están en el rango [0, 1]
    if sample_images.max() <= 1.0 and sample_images.min() >= 0.0:
        return False  # Los datos ya están normalizados
    return True  # Los datos no están normalizados
#############################################################################

#############################################################################
def wrangle_data(data_directory, split, encoding, image_size, batch_size=32):
    if verificar_normalizacion(data_directory, split, image_size):
        scale_factor = 1 / 255.0
        print('Necesita normalizacion, verificar Wrangle Data')
    else:
        scale_factor = None
        print('No necesita normalizacion, verificar Wrangle Data')


    train_datagen = ImageDataGenerator(
        rescale=scale_factor, # Ojo!!!! nunca olvidarse
        horizontal_flip=True,
        # vertical_flip=True,
        zoom_range=0.2,
        rotation_range=30,
        # width_shift_range=0.2,
        # height_shift_range=0.2,
        # brightness_range=[0.8, 1.2],
        # shear_range=0.2,
        fill_mode='nearest'
    )

    validation_test_datagen = ImageDataGenerator(rescale=scale_factor)

    if split == 'train':
        datagen = train_datagen
        shuffle = True
    elif split in ['validation', 'test']:
        datagen = validation_test_datagen
        shuffle = False
    else:
        raise ValueError("data_type must be one of ['train', 'validation', 'test']")

    data_flow = datagen.flow_from_directory(
        f'{data_directory}/{split}',
        target_size=image_size,
        batch_size=batch_size,
        class_mode=encoding,
        shuffle=shuffle
    )

    return data_flow
#############################################################################

### Main

In [42]:
# import zipfile
# import os

# # Descomprimir en la misma carpeta
# with zipfile.ZipFile(os.path.join(DATOS_DIR, 'fingers.zip'), 'r') as zip_ref:
#     zip_ref.extractall(DATOS_DIR)

In [52]:
# data_path_downloaded = DATOS_DIR + "fingers/"
# show_images_random(data_path_downloaded, "test")

In [53]:
data_path_downloaded

'/content/drive/MyDrive/Colab Notebooks/Redes_Neuronales/Data/fingers/'

In [54]:

batch_size = 32
image_size = (64, 64)
class_mode = 'sparse' # binary: Las etiquetas son enteros 0 o 1, categorical: Las etiquetas son vectores one-hot, sparse: Las etiquetas son enteros


#########################################################################
# Cells Pipeline
#########################################################################
# delete_folder_colab('../data')
# data_path = '../data/'
# # link= 'https://data.lhncbc.nlm.nih.gov/public/Malaria/cell_images.zip'
# link = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
# data_path_downloaded = get_data(link, data_path)
# print(data_path, '\n', data_path_downloaded, '\n')
# split_datapath(data_path_downloaded, 0.1)
# count_files_in_subfolders(data_path_downloaded)
# show_images_random(data_path_downloaded)
data_train_wrangled = wrangle_data(data_path_downloaded, 'train', class_mode, image_size, batch_size)
data_valid_wrangled = wrangle_data(data_path_downloaded, 'validation', class_mode, image_size, batch_size)
# # data_test_wrnagled = wrangle_data(data_path_downloaded, 'test', class_mode, image_size)



Necesita normalizacion, verificar Wrangle Data
Found 0 images belonging to 0 classes.


ValueError: zero-size array to reduction operation maximum which has no identity

## Ejercicio 4